In [130]:
import pandas as pd
import numpy as np
import datetime
import importlib
import pickle

import data_utils
import model_utils
import train_utils
import evaluation
import submission
importlib.reload(data_utils)
importlib.reload(model_utils)
importlib.reload(train_utils)
importlib.reload(evaluation)
importlib.reload(submission)

import torch
import torch.nn as nn
from torchtext.data import TabularDataset, Field, RawField, BucketIterator, Iterator

In [78]:
DEVICE = "cuda"
HOLDOUT = True

### read data and put in batch

In [79]:
train_data, val_data, test_data = data_utils.prep_all_data(
    device=DEVICE, use_holdout_test=False, ngram=1)

### Training 

In [80]:
importlib.reload(model_utils)
vocab = np.load("./data/1grams.npy")
m = model_utils.BaseModelNGram()
result = train_utils.train(train_data, val_data, m, device=DEVICE,
                          lr=1e-2, print_freq=5, max_epoch=100)

Epoch: 0, LR: 0.01, Train Loss: 239.5189, Val Loss: 97.5010, Val f1 0.743
Epoch: 5, LR: 0.01, Train Loss: 5.0058, Val Loss: 67.7320, Val f1 0.834
Epoch: 10, LR: 0.01, Train Loss: 3.0623, Val Loss: 83.7059, Val f1 0.819
Epoch: 15, LR: 0.001, Train Loss: 2.6288, Val Loss: 95.3713, Val f1 0.821


In [82]:
torch.save(result["trained_model"], "./data/model_checkpoints/ngram_MLP_Jan30.mdl")

In [85]:
importlib.reload(model_utils)
m = model_utils.GRU()
result = train_utils.train(train_data, val_data, m, device=DEVICE,
                          lr=1e-2, print_freq=5, max_epoch=100)

Epoch: 0, LR: 0.01, Train Loss: 321.0513, Val Loss: 260.8454, Val f1 0.518
Epoch: 5, LR: 0.01, Train Loss: 12.2597, Val Loss: 122.1696, Val f1 0.785
Epoch: 10, LR: 0.01, Train Loss: 1.9306, Val Loss: 121.0964, Val f1 0.782
Epoch: 15, LR: 0.01, Train Loss: 0.7654, Val Loss: 111.8629, Val f1 0.789
Epoch: 20, LR: 0.01, Train Loss: 0.4493, Val Loss: 111.4513, Val f1 0.796
Epoch: 25, LR: 0.01, Train Loss: 0.3529, Val Loss: 110.3826, Val f1 0.797
Epoch: 30, LR: 0.01, Train Loss: 0.2530, Val Loss: 115.8223, Val f1 0.796
Epoch: 35, LR: 0.01, Train Loss: 0.5006, Val Loss: 117.0286, Val f1 0.794
Epoch: 40, LR: 0.001, Train Loss: 0.4303, Val Loss: 117.0088, Val f1 0.800


In [87]:
torch.save(result["trained_model"], "./data/model_checkpoints/GRU_Jan30.mdl")

### Ensemble

In [131]:
train_data, val_data, test_data = data_utils.prep_all_data(
    device=DEVICE, use_holdout_test=False, ngram=1)
m_gram = torch.load("./data/model_checkpoints/ngram_MLP_Jan30.mdl")
m_emb = torch.load("./data/model_checkpoints/GRU_Jan30.mdl")
ensemble = submission.Ensemble(model1=m_gram, model2=m_emb, 
                               val_data=val_data, test_data=test_data)
df = ensemble.get_ensemble_result(submission=False)

ensemble f1 val score: 0.8939802336028755


### Generate submission file for kaggle 

In [132]:
df = ensemble.get_ensemble_result(submission=True)
ensemble.save_submission_file("test")

In [134]:
df.shape

(1084, 3)